In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 43 kB 7.6 MB/s eta 0:00:011
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=bb4ab0ced7f91b1581b0899514b390f9ee801fb69877a1ff9c5d5a0d8ca6d4ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-yeml15_9/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch

In [3]:
import transformers
from datasets import load_dataset, load_metric
from datasets import Sequence, ClassLabel
from datasets import Value

In [4]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [5]:
# dataset = load_dataset("conll2003", revision="master")

In [6]:
# dataset

In [14]:
PE_DATA_FILE = '/notebooks/ywc_token_classification/ywc_TokCL.pt'

In [15]:
pe_dataset = torch.load(PE_DATA_FILE)

In [16]:
pe_dataset

DatasetDict({
    train: Dataset({
        features: ['full_speech', 'Text', 'Part', 'Document', 'Order', 'Sentence', 'Start', 'End', 'Annotator', 'Tag', 'Component', 'Speech', 'Speaker', 'SpeakerType', 'Set', 'Date', 'Year', 'Name', 'MainTag', 'full_speech_words', 'tags_list', 'multi_class_tags', 'bio_only_tags', 'posfull_tokens_list', 'full_speech_words_proper', 'is_equal_tag', 'mc_int_tags', 'bio_int_tags', 'is_equal_tag_four'],
        num_rows: 1589
    })
    test: Dataset({
        features: ['full_speech', 'Text', 'Part', 'Document', 'Order', 'Sentence', 'Start', 'End', 'Annotator', 'Tag', 'Component', 'Speech', 'Speaker', 'SpeakerType', 'Set', 'Date', 'Year', 'Name', 'MainTag', 'full_speech_words', 'tags_list', 'multi_class_tags', 'bio_only_tags', 'posfull_tokens_list', 'full_speech_words_proper', 'is_equal_tag', 'mc_int_tags', 'bio_int_tags', 'is_equal_tag_four'],
        num_rows: 847
    })
    validation: Dataset({
        features: ['full_speech', 'Text', 'Part', 'Documen

In [13]:
pe_dataset['train']['Text'][0]

"And for the state to interfere in that decision, under whatever guise, and with whatever rationale, for the state to try to take over in that situation, and by edict, command what the individual shall do, and substitute itself for that individual's conscience, for her right to consult her rabbi, her minister, her priest, her doctor - any other counselor of her choice - I think goes beyond what we want to ever see accomplished in this country, if we really believe in the First Amendment: if we really believe in freedom of choice and the right of the individual."

In [10]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tags_list_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [11]:
pe_dataset = pe_dataset.rename_column("tags_list_int", "paragraph_tags_list_int")

In [12]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_list_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [13]:
pe_dataset = pe_dataset.rename_column("paragraph_tags_list_int", "paragraph_tags_int")

In [14]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

### my work today

In [15]:
# what if we sequence the int labels to the class labels? will it work?

In [16]:
pe_dataset['train'].features['paragraph_tags_bio_only_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [17]:
pe_dataset['train'].features['paragraph_tags_bio_only_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [18]:
pe_dataset['test'].features['paragraph_tags_bio_only_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [19]:
pe_dataset['validation'].features['paragraph_tags_bio_only_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [20]:
pe_dataset['test'].features['paragraph_tags_bio_only_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [21]:
pe_dataset['validation'].features['paragraph_tags_bio_only_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [22]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [23]:
pe_dataset["train"][0]["paragraph_tags_bio_only_int"]

[0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0]

In [24]:
pe_dataset["train"].features["paragraph_tags_bio_only_int"]

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [25]:
# seems correct

In [26]:
from transformers import AutoTokenizer

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [28]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [29]:
def tokenize_and_align_labels_hf(examples):
    tokenized_inputs = tokenizer(
        examples["paragraph_words"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["paragraph_tags_bio_only_int"]
    #print(examples["paragraph_tags"])
    #all_labels = labels_object.str2int(examples["paragraph_tags"])
    
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [30]:
label_all_tokens = True

In [31]:
tokenized_datasets = pe_dataset.map(tokenize_and_align_labels_hf, batched=True)

Parameter 'function'=<function tokenize_and_align_labels_hf at 0x7f39f6e95280> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
label_list = pe_dataset["train"].features['paragraph_tags_bio_only_int'].feature.names

In [33]:
label_list

['O', 'B-COMP', 'I-COMP']

In [34]:
pe_dataset["train"].features["paragraph_tags_bio_only_int"].feature.names

['O', 'B-COMP', 'I-COMP']

In [35]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [37]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=32,
    weight_decay=0.01,
    eval_steps=20,
    logging_steps=20
    #push_to_hub=True,
)

In [38]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [39]:
metric = load_metric("seqeval")

In [40]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [41]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [42]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: paragraph_tags_bio_only_correction, essay, paragraph, paragraph_tags_int, split, para_nr, essay_nr, paragraph_tags_bio_only_int, paragraph_words, paragraph_tags, paragraph_tags_bio_only.
***** Running training *****
  Num examples = 1091
  Num Epochs = 32
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2208


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.476400,0.398681,0.457543,0.578728,0.511050,0.846112
2,0.341500,0.280519,0.481909,0.652763,0.554473,0.894787
3,0.255500,0.281366,0.493808,0.665276,0.566859,0.891101
4,0.216100,0.267111,0.572621,0.715328,0.636069,0.903865
5,0.165600,0.268322,0.613579,0.763295,0.680297,0.911056
6,0.124000,0.281775,0.625108,0.752868,0.683065,0.908584
7,0.109300,0.311038,0.608074,0.753910,0.673184,0.905798
8,0.094500,0.322226,0.651794,0.776851,0.708849,0.906337
9,0.071400,0.352625,0.638502,0.764338,0.695776,0.902292
10,0.062600,0.362328,0.646649,0.774765,0.704934,0.911371


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: paragraph_tags_bio_only_correction, essay, paragraph, paragraph_tags_int, split, para_nr, essay_nr, paragraph_tags_bio_only_int, paragraph_words, paragraph_tags, paragraph_tags_bio_only.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: paragraph_tags_bio_only_correction, essay, paragraph, paragraph_tags_int, split, para_nr, essay_nr, paragraph_tags_bio_only_int, paragraph_words, paragraph_tags, paragraph_tags_bio_only.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: paragraph_tags_bio_only_correction, essay, paragraph, 

TrainOutput(global_step=2208, training_loss=0.07422333201933382, metrics={'train_runtime': 604.1008, 'train_samples_per_second': 57.792, 'train_steps_per_second': 3.655, 'total_flos': 2593568807103006.0, 'train_loss': 0.07422333201933382, 'epoch': 32.0})

In [43]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: paragraph_tags_bio_only_correction, essay, paragraph, paragraph_tags_int, split, para_nr, essay_nr, paragraph_tags_bio_only_int, paragraph_words, paragraph_tags, paragraph_tags_bio_only.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16


{'eval_loss': 0.5504189729690552,
 'eval_precision': 0.716376944190302,
 'eval_recall': 0.8164754953076121,
 'eval_f1': 0.763157894736842,
 'eval_accuracy': 0.9152808988764045,
 'eval_runtime': 1.8387,
 'eval_samples_per_second': 148.472,
 'eval_steps_per_second': 9.789,
 'epoch': 32.0}

In [44]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: paragraph_tags_bio_only_correction, essay, paragraph, paragraph_tags_int, split, para_nr, essay_nr, paragraph_tags_bio_only_int, paragraph_words, paragraph_tags, paragraph_tags_bio_only.
***** Running Prediction *****
  Num examples = 355
  Batch size = 16


{'COMP': {'precision': 0.752346570397112,
  'recall': 0.8296178343949044,
  'f1': 0.7890950397576675,
  'number': 1256},
 'overall_precision': 0.752346570397112,
 'overall_recall': 0.8296178343949044,
 'overall_f1': 0.7890950397576675,
 'overall_accuracy': 0.9260755965859606}

In [45]:
# task: try to do only with the BIO. then see the results.

In [46]:
predictions, labels, _ = trainer.predict(tokenized_datasets["train"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results_train = metric.compute(predictions=true_predictions, references=true_labels)
results_train

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: paragraph_tags_bio_only_correction, essay, paragraph, paragraph_tags_int, split, para_nr, essay_nr, paragraph_tags_bio_only_int, paragraph_words, paragraph_tags, paragraph_tags_bio_only.
***** Running Prediction *****
  Num examples = 1091
  Batch size = 16


{'COMP': {'precision': 0.9914893617021276,
  'recall': 0.9954606141522029,
  'f1': 0.993471019320453,
  'number': 3745},
 'overall_precision': 0.9914893617021276,
 'overall_recall': 0.9954606141522029,
 'overall_f1': 0.993471019320453,
 'overall_accuracy': 0.9994651393423443}